In [ ]:
import os
from os.path import join
from os import listdir
import pandas as pd
import numpy as np
import tqdm
import pandas as pd
import rouskinhf

## Plot showing the quality of our datasets

Compare replicates:
- R2 score between normalized DMS signals (first row of subplot)
- F1 score between replicate + RNAstructure (second row)

Then bootstrap from the DMS by adding noise proportional to the confidence interval, then predicting the structure with the noisy DMS. Compute the F1 score between the structures (third row of the subplot)

Use pri_miRNA and human_mRNA (called UTR previously). Combine them in one plot, or separate them in two columns if realy different

**Assigned to**: Yves

Use Ploty, and a white background

## Build dataset (no need to re-run)

In [ ]:
# load refs from HF
refs = list(rouskinhf.get_dataset('human_mRNA').keys()) + list(rouskinhf.get_dataset('pri_miRNA').keys())

# load reps data from local
pri = pd.read_feather('pri_miRNA_normalized.feather').drop(columns=['index'])
pri['dataset'] = 'Pri-miRNA'
mrna = pd.read_feather('mRNA_normalized.feather').drop(columns=['index'])
mrna['dataset'] = 'Human mRNA'
df = pd.concat([pri, mrna])

# keep only reps that are in HF
df = df[df['replicate'] != 'Untreated']
df = df[df['reference'].isin(refs)].reset_index(drop=True)

# count the number of reference per sample
df['number_of_replicates'] = df.groupby(['reference', 'dataset', 'plate'])['reference'].transform('count')

# Count the amount of available reps per dataset
ref_per_sample_per_dataset = {}
for dataset in df['dataset'].unique():
    loc_df = df[df['dataset']==dataset]
    ref_per_sample_per_dataset[dataset] = {k[0]: v for k, v in dict(loc_df.value_counts(['number_of_replicates']).sort_index()).items()} 

# drop the reference with less than 2 sample 
l = len(df)
df = df[df['number_of_replicates'] == 2]
print("drop {}/{} references".format(l - len(df), l))

# Run RNAstructure
rnastructure_path = '/Users/yvesmartin/lib/RNAstructure/exe'

def file_name(stage): 
    return f'{stage}/{file_name.ref}_{file_name.replicate}.{stage}'

for stage in ['fasta', 'shape', 'ct', 'dot']:
    os.makedirs(stage, exist_ok=True)
    
def predict_structure(seq, ref, rep, dms=None):
    file_name.ref = ref
    file_name.replicate = rep
    os.system(f'echo ">{ref}\n{seq}" >{file_name("fasta")}')
    if dms is None:
        dms = np.zeros(len(seq))# * -1000.
    dms = pd.DataFrame({'dms': [float(d) for d in dms], 'seq': list(seq)})
    dms.index = dms.index + 1
    dms = dms[dms['seq'].isin(['A', 'C'])]['dms']
    dms.to_csv(file_name('shape'), sep='\t', header=False)
    os.system(f"{rnastructure_path}/fold {file_name('fasta')} {file_name('ct')} --DMS {file_name('shape')}")
    os.system(f"{rnastructure_path}/ct2dot {file_name('ct')}  -1 {file_name('dot')}" )
    with open(file_name('dot')) as f:
        lines = f.readlines()
    energy = float(lines[0].split()[2])
    structure = lines[2].strip()
    return energy, structure


for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    seq, ref, rep, dms = row[['sequence', 'reference', 'replicate', 'sub_rate']]
    energy, structure = predict_structure(seq, ref, rep, dms)
    df.loc[i, 'energy'] = energy
    df.loc[i, 'structure'] = structure
    
# read RNAstructure output
def read_dot(dot_file):
    with open(dot_file, 'r') as f:
        lines = f.readlines()
    energy, structure = [], []

    def get_energy(line):
        return float(line.split(' ')[2].strip())

    energy.append(get_energy(lines[0]))
    structure.append(lines[2].strip())
    
    for line in lines[3:]:
        if line.startswith('>ENERGY'):
            energy.append(get_energy(line))
        else:
            structure.append(line.strip())
    
    assert len(energy) == len(structure)
    return energy, structure

rna_pred = []
for file in os.listdir('dot'):
    energies, structures = read_dot(f'dot/{file}')
    for idx, (s, e) in enumerate(zip(structures, energies)):
        rna_pred.append({
            'reference': file[:-6],
            'structure': s,
            'energies': e,
            'replicate': file.split('_')[-1].split('.')[0],
            'rank': idx
        })
    
rna_pred = pd.DataFrame(rna_pred)

# merge with df
df = pd.merge(df, rna_pred, on=['reference', 'replicate'], how='inner')

df.to_feather('data_quality.feather')

## Plot replicates distribution per dataset

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


ref_per_sample_per_dataset = pd.DataFrame(ref_per_sample_per_dataset)
ref_per_sample_per_dataset.index = ['No replicate', '1 replicate']
fig = make_subplots(rows=1, cols=2, subplot_titles=ref_per_sample_per_dataset.columns, specs=[[{'type':'domain'}, {'type':'domain'}]])

for i, col in enumerate(ref_per_sample_per_dataset.columns):
    fig.add_trace(
        go.Pie(
            labels=ref_per_sample_per_dataset.index, 
            values=ref_per_sample_per_dataset[col],
            name=col,
            textinfo='label+percent',
            textposition='auto',
            rotation=270,
    ), row=1, col=i+1)

fig.update_layout(
    title_text="Number of replicates per reference in each dataset",
    width=800,
)
fig.show()

# Load data

In [ ]:
import pandas as pd
import numpy as np
import tqdm

df_all = pd.read_feather('data_quality.feather')
df = df_all.sort_values('energies').groupby(['reference', 'replicate']).first().reset_index()


## Compute scores

In [ ]:
import plotly.graph_objects as go

def dot2bool(dot):
    return np.array([1 if c == '.' else 0 for c in dot])

def compute_score_between_replicates(df, score):
    scores = []
    for (ref, plate), group in tqdm.tqdm(df.groupby(['reference', 'plate']), total=len(df['reference'].unique())):
        A = group[group['replicate'] == 'A']
        B = group[group['replicate'] == 'B']
        dataset = group['dataset'].values[0]
        scores.append({
            'reference': ref,
            'plate': plate,
            'replicate': 'A',
            score.name: score(A, B),
            'dataset': dataset
        })
        scores.append({
            'reference': ref,
            'plate': plate,
            'replicate': 'B',
            score.name: score(B, A),
            'dataset': dataset
        })
    
    return pd.DataFrame(scores)

def plot_score_histogram(df, scores, score, min_X):
    fig = go.Figure()
    for name, dataset in scores.groupby('dataset'):
        fig.add_trace(go.Histogram(x=dataset[score.name], name=name, 
                            xbins=dict(start=min_X, end=1, size=0.05),
                            ))

    fig.update_layout(
        title='{} score (N={}, # of {} scores < {} = {}) for df: {}'.format(score.name, len(scores), score.name, min_X, len(scores[scores[score.name] < min_X]), df.custom_name),
        xaxis_title=score.name,
        yaxis_title='count',
        bargap=0.2,
        bargroupgap=0.1,
        xaxis_range=[min_X, 1],
    )
    return fig

def violin_plot(df, scores, score, min_X):  
    fig = go.Figure()
    for name, dataset in scores.groupby('dataset'):
        fig.add_trace(go.Violin(x=dataset[score.name], name=name + ' (N={})'.format(len(dataset)),
                            box_visible=True,
                            meanline_visible=True,
                            ))

    fig.update_layout(
        title='{} score distribution between replicates'.format(score.name),
        xaxis_title=score.name + ' score',
        # yaxis_title='dataset',
        bargap=0.2,
        bargroupgap=0.1,
        xaxis_range=[min_X, 1],
        showlegend=False,
        width=800,    
        paper_bgcolor='white',  # Background color of the entire plot
        plot_bgcolor='white',  # Background color of the plot area
        # add a frame
        margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins
        xaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='lightgrey',
            mirror=True,
            showgrid=False,
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='lightgrey',
            mirror=True,
            showgrid=False,
            gridcolor='white',
            gridwidth=2,
        ),
        font=dict(
        size=18,
    )
    )
    
    return fig
    
def normalize(x, mask=None):
    x = np.array(x)
    mask = x != -1000. if mask is None else mask
    per90 = np.percentile(x, 90)
    y = np.clip(x[mask] / per90, 0, 1)
    x[mask] = y
    return x


### Add RNAstructure normalized structure

In [ ]:
from rnastructure import RNAstructure
rna = RNAstructure()

def normalized_structure_prediction(row):
    dms = normalize(row['sub_rate'])
    seq = row['sequence']
    return rna.predictStructure(seq, dms=dms)

for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    df.loc[i, 'structure_normalized'] = normalized_structure_prediction(row)
    
df.to_feather('data_quality.feather')

### AUROC score


In [ ]:
from sklearn.metrics import roc_auc_score

def auroc(row):
    sig = row['sub_rate']
    mask = sig != -1000.
    sig = sig[mask]
    structure = dot2bool(row['structure'])[mask]
    return roc_auc_score(structure, sig)

df_all['auroc'] = df_all.apply(lambda x: auroc(x), axis=1)
df_best = df_all.sort_values('auroc', ascending=False).groupby(['reference', 'plate','replicate']).first().reset_index()

df.custom_name = 'lowest predicted energy'
df_best.custom_name = 'best auroc'

### R2 score

In [ ]:
from sklearn.metrics import r2_score

def r2_score_obj(x, y):
    A, B = x['sub_rate'].values[0], y['sub_rate'].values[0]
    mask = (A != -1000.) & (B != -1000.)
    A, B = A[mask], B[mask]
    return r2_score(A, B)

r2_score_obj.name = 'R2'
scores = compute_score_between_replicates(df, r2_score_obj)
# style like a paper
fig = violin_plot(df, scores, r2_score_obj, 0.)
# style the plot like a pape
    

fig.show()
# violin_plot(df_best, compute_score_between_replicates(df_best, r2_score_obj), r2_score_obj, 0.).show()


### F1 score

In [ ]:
# import f1_score
from sklearn.metrics import f1_score as f1_score_sklearn

import rnastructure
rna = rnastructure.RNAstructure()

def f1_score(x0, x1):
    x0, x1 = x0['structure'].values[0], x1['structure'].values[0]
    return f1_score_sklearn(dot2bool(x0), dot2bool(x1))

f1_score.name = 'f1'

fig = violin_plot(df, compute_score_between_replicates(df, f1_score), f1_score, 0.).update_layout(width=800)

# make text bigger
fig

In [ ]:
def pearson_score(x, y):
    A, B = x['sub_rate'].values[0], y['sub_rate'].values[0]
    mask = (A != -1000.) & (B != -1000.)
    A, B = A[mask], B[mask]
    return np.corrcoef(A, B)[0, 1]

pearson_score.name = 'pearson'


violin_plot(df, compute_score_between_replicates(df, pearson_score), pearson_score, 0.).update_layout(width=800).show()


In [ ]:
def coeff_of_determination(x, y):
    A, B = x['sub_rate'].values[0], y['sub_rate'].values[0]
    mask = (A != -1000.) & (B != -1000.)
    A, B = A[mask], B[mask]
    return np.corrcoef(A, B)[0, 1]**2

coeff_of_determination.name = 'coeff_of_determination'
violin_plot(df, compute_score_between_replicates(df, coeff_of_determination), coeff_of_determination, 0.).update_layout(width=800).show()

## Bootstrapping on F1 score

In [ ]:
df_pri_miRNA = pd.DataFrame.from_dict(rouskinhf.get_dataset('pri_miRNA'), orient='index').reset_index().rename(columns={'index':'reference'})
df_pri_miRNA['dataset'] = 'pri_miRNA'

# load human_mRNA from HF
df_human_mRNA = pd.DataFrame.from_dict(rouskinhf.get_dataset('human_mRNA'), orient='index').reset_index().rename(columns={'index':'reference'})
df_human_mRNA['dataset'] = 'human_mRNA'

df_hf = pd.concat([df_pri_miRNA, df_human_mRNA])

# get the number of reads from df
df_hf['reads'] = df_hf['reference'].apply(lambda x: df[df['reference'] == x]['reads'].values[0])

df_hf

In [ ]:
# define a bootstrapping function using a binomial distribution

import numpy as np
import tqdm

n_bootstrap = 10

def bootstrap(n, p, N=n_bootstrap):
    l = len(p)
    n = (np.ones(l) * n).astype(int)
    return np.random.binomial(n, p, (N, l)) / n

f1_scores_total = {}
r2_scores_total = {}
for _, line in tqdm.tqdm(df_hf.iterrows(), total=len(df_hf)):
    reference, sequence, dms, dataset, reads = line[['reference', 'sequence', 'dms', 'dataset', 'reads']]
    dms = np.array(dms) 
    f1_scores_local = []
    r2_scores_local = []
    dms_boot = bootstrap(reads, dms)
    for dms in dms_boot:
        pred = rna.predictStructure(sequence, dms=dms)
        f1_scores_local.append(f1_score_sklearn(dot2bool(pred), dot2bool(pred)))
        r2_scores_local.append(r2_score(dms, pred))
        
    f1_scores_total[reference] = f1_scores_local
    r2_scores_total[reference] = r2_scores_local
    
df_hf['f1_scores'] = df_hf['reference'].apply(lambda x: f1_scores_total[x])
df_hf['r2_scores'] = df_hf['reference'].apply(lambda x: r2_scores_total[x])

# dump the data
df_hf.to_feather('data_quality_bootstrapping.feather')